# Keras Tuner: Optimización de hiperparámetros

Keras Tuner es una librería que ayuda a elegir el conjunto óptimo de hiperparámetros de manera automática. El proceso de seleccionar el conjunto correcto de hiperparámetros para un algoritmo de aprendizaje automático ajuste de hiperparámetros o *hypertuning*.
Existen dos tipos de hiperparámetros:
- **Hiperparámetros de modelo**: Influyen en la selección del modelo, como el número y el ancho de las capas ocultas.

- **Hiperparámetros de algoritmo**: Influyen en la velocidad y la calidad del algoritmo de aprendizaje, p.ejem. la tasa de aprendizaje para el descenso de gradiente estocástico (SGD) y el número de vecinos más cercanos para un clasificador KNN.

### Instalamos Keras Tuner y cargamos dataset fashion MNIST

In [ ]:
!pip freeze

In [ ]:
!pip install -q -U keras-tuner
import kerastuner as kt

In [ ]:
from tensorflow.keras.datasets import fashion_mnist
(img_train, label_train), (img_test, label_test) = fashion_mnist.load_data()
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

### Creamos la función que define la topología de red y compila el modelo

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

def model_builder(hp):
  model = Sequential()
  model.add(Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(Dense(units = hp_units, activation = 'relu'))
  model.add(Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

  model.compile(optimizer = Adam(learning_rate = hp_learning_rate),
                loss = SparseCategoricalCrossentropy(from_logits = True),
                metrics = ['accuracy'])

  return model

### Instanciamos el objeto optimizador

In [5]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

### Callback para limpiar las salidas de los entrenamientos al finalizar cada paso

In [6]:
import tensorflow as tf
import IPython

class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

### Llevamos a cabo la optimización de hiperparámetros y nos quedamos con el mejor resultado

In [7]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 23s]
val_accuracy: 0.8773000240325928

Best val_accuracy So Far: 0.8930000066757202
Total elapsed time: 00h 16m 16s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 480 and the optimal learning rate for the optimizer
is 0.001.



# MLflow
Se trata de una plataforma *open source* para la gestión del ciclo de vida de un modelo de *machine learning*. MLflow ofrece cuatro componentes:
- ***MLflow Tracking***: Almacenamiento inteligente y monitorización de experimentos (código, datos, configuración y resultados).
- ***MLflow Projects***: Paquete de código que permite reproducir cualquier formato de código en cualquier plataforma de trabajo.
- ***MLflow Models***: Puesta en marcha de modelo en producción en diversos entornos de servicios.
- ***Model Registry***: Almacena, anota, descubre y gestiona modelos en un repositorio centralizado.

Demos un paseo por la documentación y ejemplos de la librería: https://mlflow.org/